# LLM Client Usage Demo

This notebook demonstrates how to use the `llm_client.py` module to create and interact with various LLM providers (OpenAI, Deepseek, Gemini) via the `LLMFactory`.

## 2. Import LLM Client Classes

In [1]:
import os
import json
from dotenv import load_dotenv

# ensure repo root is on path
import sys
sys.path.append(os.path.abspath(os.path.join("..", "")))

from src.core.llm_client import ModelProvider, LLMFactory, OpenAIClient, DeepseekClient, GeminiClient

## 3. Configure Environment Variables

In [2]:
# either set in this cell (not recommended for secrets) or create a .env file
load_dotenv()

# example assignments (uncomment and fill with real keys if desired):
# os.environ['OPENAI_API_KEY'] = 'your-openai-key'
# os.environ['DEEPSEEK_API_KEY'] = 'your-deepseek-key'
# os.environ['GEMINI_API_KEY'] = 'your-gemini-key'

print("keys loaded?", 
      'OPENAI_API_KEY' in os.environ, 
      'DEEPSEEK_API_KEY' in os.environ, 
      'GEMINI_API_KEY' in os.environ)

keys loaded? True True True


## 4. Instantiate Clients with Different Providers

In [4]:
# create clients via the factory
openai_client = LLMFactory.create_client(ModelProvider.OPENAI, temperature=0.3)
deepseek_client = LLMFactory.create_client(ModelProvider.DEEPSEEK, model_name='deepseek-chat', temperature=0.5)
gemini_client = LLMFactory.create_client(ModelProvider.GOOGLE, model_name='gemini-flash-latest', temperature=0.7)

print(openai_client, deepseek_client, gemini_client)

<src.core.llm_client.OpenAIClient object at 0x1218e2710> <src.core.llm_client.DeepseekClient object at 0x11237f9d0> <src.core.llm_client.GeminiClient object at 0x112b50f50>


## 4.1 Notes on warnings

## 5. Generate Text with Each Client

You may see output like object memory addresses or deprecation warnings. The memory addresses (e.g. `<src.core.llm_client.OpenAIClient object at ...>`) are just Python's default `repr` for the instances; the above change prints a friendly summary instead.

Tqdm warnings can be resolved by installing and enabling `ipywidgets` in your Jupyter environment. The Google package deprecation message indicates you should migrate from `google.generativeai` to `google.genai` when updating your dependencies.

In [ ]:
prompt = "Please give me a two‑sentence inspirational quote about learning with AI.  Make it at least 50 words."
system = "You are a helpful assistant."

# print("--- OpenAI ---")
# print(openai_client.generate_text(prompt, system_prompt=system))
# print(openai_client.generate_text(prompt, system_prompt=system, temperature=0.5))

print("\n--- Deepseek ---")
print(deepseek_client.generate_text(prompt, system_prompt=system, temperature=0.5))

print("\n--- Gemini ---")
print(gemini_client.generate_text(prompt, system_prompt=system, temperature=0.5))

--- OpenAI ---

--- Deepseek ---
Embrace the boundless curiosity of artificial intelligence as your companion, for it transforms the vast, uncharted ocean of human knowledge into a navigable river, lighting the path with instant insights and revealing patterns once hidden in the dark. Let this powerful synergy ignite a lifelong fire within you, where every question becomes a doorway and every interaction with a machine deepens your own uniquely human capacity for wonder, creativity, and profound understanding.
When we integrate artificial intelligence into our educational journey, we transform the vast ocean of global information into a personalized stream of profound insight and endless discovery. This symbiotic relationship between human curiosity and machine intelligence serves as a luminous beacon, guiding us toward a new era where the potential for growth is limited only by the reach of our imagination.


## 6. Generate JSON with Each Client

In [7]:
# define a simple schema
schema = {
    "type": "object",
    "properties": {
        "message": {"type": "string"},
        "length": {"type": "integer"}
    },
    "required": ["message", "length"]
}

json_prompt = "Provide a JSON object containing an encouraging message and its character count."

for client, name in [(openai_client, 'OpenAI'), (deepseek_client, 'Deepseek'), (gemini_client, 'Gemini')]:
    print(f"--- {name} JSON response ---")
    try:
        result = client.generate_json(json_prompt, schema=schema)
        print(result)
    except Exception as e:
        print("error", e)

--- OpenAI JSON response ---
{'message': 'Believe in yourself and all that you are. Know that there is something inside you that is greater than any obstacle.', 'length': 111}
--- Deepseek JSON response ---
{'message': "You're doing great! Every step forward, no matter how small, is progress. Keep going!", 'length': 78}
--- Gemini JSON response ---
{'message': 'You are capable of achieving great things.', 'length': 42}


## 7. Handle Errors and Validations

In [ ]:
# missing key example
os.environ.pop('OPENAI_API_KEY', None)
try:
    LLMFactory.create_client(ModelProvider.OPENAI)
except ValueError as e:
    print("Expected error for missing key:", e)

# JSON parsing failure simulation: send bad prompt
os.environ['OPENAI_API_KEY'] = 'dummy'
class DummyClient(OpenAIClient):
    def generate_text(self, prompt, max_tokens=None, system_prompt=None):
        return "not a json"

bad_client = DummyClient(api_key='abc', model_name='gpt-4o')
try:
    bad_client.generate_json('anything', schema={})
except ValueError as e:
    print("Caught JSON parse error:", e)